<a href="https://colab.research.google.com/github/aworklab/everydayPython/blob/main/ex_20220927.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 오늘의 목표
* Customer analysis

In [1]:
import pandas as pd 
import numpy as np 
import plotnine as gg
from plotnine import ggplot, aes 
from datetime import datetime, date, time, timezone, timedelta 
import os
import re
import matplotlib.pyplot as plt

# Customer analysis

In [2]:
os.listdir()

['.config',
 'KPMG_VI_New_raw_data_update_final.xlsx',
 'scanner_data.csv',
 'ecommerce-data.csv',
 'online_retail_II.xlsx',
 'KPMG_VI_New_raw_data_update_final.csv',
 'test.csv',
 'sample_data']

In [3]:
df = pd.read_csv('scanner_data.csv')
df.head()

,Unnamed: 0,Date,Customer_ID,Transaction_ID,SKU_Category,SKU,Quantity,Sales_Amount
0,1,02/01/2016,2547,1,X52,0EM7L,1.0,3.13
1,2,02/01/2016,822,2,2ML,68BRQ,1.0,5.46
2,3,02/01/2016,3686,3,0H2,CZUZX,1.0,6.35
3,4,02/01/2016,3719,4,0H2,549KK,1.0,5.59
4,5,02/01/2016,9200,5,0H2,K8EHH,1.0,6.88


In [4]:
df.drop(['Unnamed: 0'], axis = 1, inplace = True)
df.columns = [a.lower() if a != 'Customer_ID' else 'id' for a in df.columns]
df.head()

,date,id,transaction_id,sku_category,sku,quantity,sales_amount
0,02/01/2016,2547,1,X52,0EM7L,1.0,3.13
1,02/01/2016,822,2,2ML,68BRQ,1.0,5.46
2,02/01/2016,3686,3,0H2,CZUZX,1.0,6.35
3,02/01/2016,3719,4,0H2,549KK,1.0,5.59
4,02/01/2016,9200,5,0H2,K8EHH,1.0,6.88


In [6]:
df['date'] = pd.to_datetime(df['date'])
df['week'] = df['date'] - df['date'].dt.weekday.astype('timedelta64[D]')
df['month'] = df['date'].to_numpy().astype('datetime64[M]')
df['rev'] = df.quantity * df.sales_amount
df = df.groupby(['id','month','date'])['rev'].sum().reset_index()
df.head()

,id,month,date,rev
0,1,2016-01-01,2016-01-22,16.29
1,2,2016-03-01,2016-03-24,7.77
2,2,2016-06-01,2016-06-19,15.00
3,3,2016-02-01,2016-02-01,19.08
4,4,2016-09-01,2016-09-11,26.60


In [13]:
info_m = df.groupby(['id','month'])['rev'].sum().reset_index().rename(columns = {'rev':'money'})
info_m = pd.pivot_table(info_m, index = 'id', columns = 'month', values = 'money').reset_index()
info_m = pd.melt(info_m, id_vars = 'id', var_name = 'month', value_name = 'money').reset_index()
info_m = info_m.sort_values(['id','month'], ascending = [1,0]).groupby(['id']).head(3)
info_m = info_m.groupby(['id'])['money'].sum().reset_index()
info_m.head()

,id,money
0,1,0.00
1,2,0.00
2,3,0.00
3,4,6.69
4,5,0.00


In [15]:
info_f = df.groupby(['id','month'])['rev'].count().reset_index().rename(columns = {'rev':'freq'})
info_f = pd.pivot_table(info_f, index = 'id', columns = 'month', values = 'freq').reset_index()
info_f = pd.melt(info_f, id_vars = 'id', var_name = 'month', value_name = 'freq').reset_index()
info_f = info_f.sort_values(['id','month'], ascending = [1,0]).groupby(['id']).head(3)
info_f = info_f.groupby(['id'])['freq'].sum().reset_index()
info_f.head()

,id,freq
0,1,0.0
1,2,0.0
2,3,0.0
3,4,1.0
4,5,0.0


In [16]:
date_max = df['date'].max() + pd.offsets.MonthEnd()
date_max

Timestamp('2017-01-31 00:00:00')

In [19]:
info_r = df.groupby(['id']).agg({'date' : lambda x : (date_max - x.max())}).reset_index().rename(columns = {'date':'recency'})
info_r['recency'] = info_r['recency'].dt.days
info_r.head()

,id,recency
0,1,375
1,2,226
2,3,365
3,4,85
4,5,151


In [20]:
info = pd.merge(pd.merge(info_r, info_f, 'left', 'id'), info_m, 'left', 'id')
info = info.assign(
    r = pd.qcut(info['recency'], 5, list('12345')),
    f = pd.qcut(info['freq'].rank(method = 'first'), 5, list('54321')),
    m = pd.qcut(info['money'].rank(method = 'first'), 5, list('54321'))
)
info['fm'] = info.f.astype(str) + info.m.astype(str)
info.head()

,id,recency,freq,money,r,f,m,fm
0,1,375,0.0,0.00,5,5,5,55
1,2,226,0.0,0.00,4,5,5,55
2,3,365,0.0,0.00,5,5,5,55
3,4,85,1.0,6.69,2,3,2,32
4,5,151,0.0,0.00,3,5,5,55
